In [1]:
import pandas as pd
import numpy as np
from datetime import date


from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

driver_path = '/Users/zacharywong/Downloads/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service = service)

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-42be4f3c9805.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

#from email import encoders
import email, ssl, smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase

from datetime import date

#relative path to dataframe folder
pathtoDFolder = '../IntermediateDataFrames/'


In [2]:
def readinValue(spreadsheet_id, cellLocation):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(0)
    value = worksheet.acell(cellLocation).value
    return value

In [3]:
def changeCellValue(spreadsheet_id, cellLocation, value):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(0)
    worksheet.update(cellLocation, value)


In [4]:
spreadsheet_id = '1WYvfPFW6n2hOCZ-2_pTT0hjJOj5vGHbwjLnjzDqXhSM'
cellLocationLimit = 'B10'
cellLocationWeight = 'B11'
SongLimit = int(readinValue(spreadsheet_id, cellLocationLimit))

# Shazam only has data for 45 songs for the WoonSocket, Rhode Island Top Charts even though all other cities have 50 songs

maxSongLimit = 45
if SongLimit > maxSongLimit: 
    SongLimit = maxSongLimit
    changeCellValue(spreadsheet_id, cellLocationLimit, maxSongLimit)


UnrankedSongChangedWeight = int(readinValue(spreadsheet_id, cellLocationWeight))
SongLimit

40

In [5]:
url = 'https://www.shazam.com/charts/top-200/united-states'
xpath = '//div[@class="download-csv"]/a'
filename = 'USTopSongs.csv'
path = pathtoDFolder + 'USTopSongs.csv'
driver.get(url)
waittime = 20
element = WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, xpath)))
csvlink = ''

try: 
    csv = driver.find_element(By.XPATH, xpath)
    csvlink = csv.get_attribute('href')


except: 
    csv = driver.find_element(By.XPATH, xpath)
    csvlink = csv.get_attribute('href')

df = pd.read_csv(csvlink, skiprows=2)
columns = df.columns.tolist()
columns = columns[1:3] + columns[0:1]
dfUSTopSongs = df[columns].rename(columns={'Rank':'Rank in Top US Chart', 'Title': 'Song Name'}).head(SongLimit)
dfUSTopSongs.to_csv(path, index=False)
dfUSTopSongs

,Artist,Song Name,Rank in Top US Chart
0,Elton John & Dua Lipa,Cold Heart (PNAU Remix),1
1,Latto,Big Energy,2
2,Muni Long,Hrs and Hrs,3
3,Kodak Black,Super Gremlin,4
4,Billie Eilish,Billie Bossa Nova,5
5,Glass Animals,Heat Waves,6
6,GAYLE,abcdefu,7
7,NEIKED & Mae Muller Feat. Polo G,Better Days,8
8,Ed Sheeran,Shivers,9
9,Gerry Rafferty,Right Down The Line,10


In [6]:
xpath = '//select[@data-shz-type="city"]'
driver.get(url)
firstindex = 0
baseurl = 'https://www.shazam.com/services/charts/csv/top-50/united-states/'

driver.get(url)
element = WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, xpath)))

try:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)
    
except:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)

#cities
citylist = []

for city in dropdown.options:
    citynormalized = city.text.replace(',', '').replace(' ', '-')
    citylist.append(citynormalized)
citylist.pop(firstindex)
len(citylist)

#get csv links for each city in dictionary 

cityURL = {}
for city in citylist:
    csvurl = baseurl + str(city)
    cityURL[city] = csvurl

cityURL
driver.quit()
    

In [7]:
#Create Panda Dataframe
columnsPart = ["Song Name", "Artist"]
columns = columnsPart + citylist
dfCityTopSongs = pd.DataFrame(columns = columns)
dfCityTopSongs

,Song Name,Artist,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma


In [8]:
# A Helper Function to fill in Song Rankings in the dfCityTopSongs dataframe 

def updateSongRank(dfCity, SongIndex, dfCityTopSongs, SongLimit, CityName):
    
    #Append top songs to the CityTop20 Dataframe 
    SongRank = 0
    while SongRank < SongLimit: 
        SongName =  dfCity.at[SongRank, 'Title']
        SongArtist =  dfCity.at[SongRank, 'Artist']
        
        # append new song/artist to dataframe if not already inside 
        if SongName not in dfCityTopSongs['Song Name'].unique():
            dfCityTopSongs = dfCityTopSongs.append({"Song Name": SongName, "Artist": SongArtist}, ignore_index = True)
            global SongIndexCounter
            SongIndex.update({SongName: SongIndexCounter})  
            SongIndexCounter += 1
            #print(SongName)
            
        # fill in the value (rank) for the song for the specific city if not the same as previous day
        # update rank for next song
        if (dfCityTopSongs.at[SongIndex.get(SongName), CityName] != SongRank+1):
            dfCityTopSongs.at[SongIndex.get(SongName), CityName] = SongRank+1
        SongRank += 1
        
    #print(SongIndex)
    #Reset song rank
    return dfCityTopSongs

In [9]:
# Keeps track of the song and its index in the dfCityTopSongs dataframe 
SongIndex = {}
SongIndexCounter = 0
filename = 'CityTopSongs.csv'
path = pathtoDFolder + filename

# Loop through each city 
for cityName in citylist: 
    cityurl = cityURL.get(cityName)
    dfCity = pd.read_csv(cityurl, skiprows = 2)
    dfCityTopSongs = updateSongRank(dfCity, SongIndex, dfCityTopSongs, SongLimit, cityName)

dfCityTopSongs.to_csv(path, index = False)

In [10]:
# Set indexes on the City Chart dataframe to Song Name and Artist 
# Set indexes on the USTop50 DataFrame to Song Name and Artist

dfCityTopSongs = dfCityTopSongs.set_index(['Song Name', 'Artist'])
dfUSTopSongs = dfUSTopSongs.set_index(['Song Name', 'Artist'])
dfUSTopSongs

,,Rank in Top US Chart
Song Name,Artist,
Cold Heart (PNAU Remix),Elton John & Dua Lipa,1
Big Energy,Latto,2
Hrs and Hrs,Muni Long,3
Super Gremlin,Kodak Black,4
Billie Bossa Nova,Billie Eilish,5
Heat Waves,Glass Animals,6
abcdefu,GAYLE,7
Better Days,NEIKED & Mae Muller Feat. Polo G,8
Shivers,Ed Sheeran,9


In [11]:
# Importance Score Calculation
# 1. add up all rankings in each city (songs that didn't break into a city are all counted as 21 rank)
# 2. Divide by number of cities 
# 3. Thus, most important songs have lowest Importance Score 

NumberofCities = 202
NonRankedWeight = NumberofCities + UnrankedSongChangedWeight

dfCityTopSongs['Sum of Rankings'] = dfCityTopSongs.sum(axis = 1)
dfCityTopSongs['Number of Cities without Rank'] = dfCityTopSongs.isna().sum(axis = 1)
def calculateImportance(row):
    row['Importance Score'] = (row['Sum of Rankings'] + (row['Number of Cities without Rank'] * NonRankedWeight))/NumberofCities
    return row
dfCityTopSongs = dfCityTopSongs.apply(calculateImportance, axis = 'columns')
dfCityTopSongs['Importance Score'] = dfCityTopSongs['Importance Score'].round(2)
#Clean up dataframe 

del(dfCityTopSongs['Sum of Rankings'])
del(dfCityTopSongs['Number of Cities without Rank'])

dfCityTopSongs.head()


,,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,Anchorage,Asheville,...,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma,Importance Score
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
Big Energy,Latto,1.0,1.0,4.0,3.0,5.0,2.0,2.0,1.0,2.0,NaN,...,4.0,10.0,5.0,3.0,NaN,6.0,6.0,12.0,2.0,23.76
Heat Waves,Glass Animals,2.0,NaN,3.0,10.0,30.0,3.0,29.0,6.0,6.0,27.0,...,10.0,14.0,3.0,11.0,NaN,14.0,5.0,24.0,3.0,36.31
Cold Heart (PNAU Remix),Elton John & Dua Lipa,3.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,13.0,...,13.0,4.0,1.0,2.0,NaN,1.0,1.0,1.0,1.0,15.89
I Should Live In Salt,The National,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,5.0,...,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,183.42
Right Down The Line,Gerry Rafferty,5.0,NaN,8.0,36.0,8.0,12.0,17.0,4.0,9.0,3.0,...,37.0,3.0,24.0,30.0,NaN,5.0,7.0,36.0,22.0,55.62


In [12]:
# Add in column counting how many cities the song broke Top 20 Charts
ColumnName = 'Number of Cities where Song Broke Top Chart'
Series = dfCityTopSongs.count(axis = 'columns')-1
Series = Series.rename(ColumnName)
dfCityTopSongs = pd.concat([dfCityTopSongs, Series], axis = 1)
dfCityTopSongs


,,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,Anchorage,Asheville,...,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma,Importance Score,Number of Cities where Song Broke Top Chart
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
Big Energy,Latto,1.0,1.0,4.0,3.0,5.0,2.0,2.0,1.0,2.0,NaN,...,10.0,5.0,3.0,NaN,6.0,6.0,12.0,2.0,23.76,186
Heat Waves,Glass Animals,2.0,NaN,3.0,10.0,30.0,3.0,29.0,6.0,6.0,27.0,...,14.0,3.0,11.0,NaN,14.0,5.0,24.0,3.0,36.31,177
Cold Heart (PNAU Remix),Elton John & Dua Lipa,3.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,13.0,...,4.0,1.0,2.0,NaN,1.0,1.0,1.0,1.0,15.89,190
I Should Live In Salt,The National,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,5.0,...,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,183.42,31
Right Down The Line,Gerry Rafferty,5.0,NaN,8.0,36.0,8.0,12.0,17.0,4.0,9.0,3.0,...,3.0,24.0,30.0,NaN,5.0,7.0,36.0,22.0,55.62,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dumb Enough,Feni Feat. Yoflem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,211.01,1
Me Meti En El Ruedo (En Vivo),Luis R Conriquez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,38.0,NaN,32.0,210.25,2
Alien Blues,Vundabar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,211.06,1


In [13]:
# Concatenate the two dataframes into 1 dataframe

df = pd.concat([dfUSTopSongs, dfCityTopSongs], axis=1)
df.head()


,,Rank in Top US Chart,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,Anchorage,...,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma,Importance Score,Number of Cities where Song Broke Top Chart
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
'Til You Can't,Cody Johnson,NaN,9.0,NaN,NaN,30.0,NaN,40.0,NaN,NaN,NaN,...,NaN,30.0,NaN,NaN,NaN,NaN,25.0,NaN,151.44,65.0
(I Just) Died In Your Arms,Cutting Crew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.80,7.0
151 Rum,JID,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.13,1.0
2 On,Tinashe Feat. Schoolboy Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210.99,1.0
20 Min,Lil Uzi Vert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.01,1.0


In [14]:
# Add Another column that shows if the song broke US Top 20 Chart
#index = ['Song Name', 'Artist']
df = df.reset_index()
ColumnName = 'Broke US Top Chart'
Series = pd.Series(np.where(pd.isna(df['Rank in Top US Chart']), False, True))
Series = Series.rename(ColumnName)
df = pd.concat([df, Series], axis = 1)
df = df.set_index(['Song Name', 'Artist'])
df
#BrokeChartDF[ColumnNames[2]] = BrokeChartList
#BrokeChartDF.set_index([ColumnNames[0], ColumnNames[1]])

#df = df.set_index()
#ColumnNames = ['Song Name', 'Artist', 'Broke US Top Chart']
##BrokeChartDF = pd.DataFrame(columns = ColumnNames)
#BrokeChartDF['Song Name'] = df['Song Name']
#BrokeChartDF


,,Rank in Top US Chart,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,Anchorage,...,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma,Importance Score,Number of Cities where Song Broke Top Chart,Broke US Top Chart
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
'Til You Can't,Cody Johnson,NaN,9.0,NaN,NaN,30.0,NaN,40.0,NaN,NaN,NaN,...,30.0,NaN,NaN,NaN,NaN,25.0,NaN,151.44,65.0,False
(I Just) Died In Your Arms,Cutting Crew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.80,7.0,False
151 Rum,JID,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.13,1.0,False
2 On,Tinashe Feat. Schoolboy Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210.99,1.0,False
20 Min,Lil Uzi Vert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.01,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Нана Ахь Са Хунда Гатдо,Ахмед Идигов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.02,1.0,False
Под крылом,Нэлли Мотяева,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.09,1.0,False
ТЫ ГОРИШЬ КАК ОГОНЬ,SLAVA MARLOW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.10,1.0,False


In [15]:
filename = 'TopSongsinUSandCity.csv'
path = pathtoDFolder + filename

#Reorder columns

columns = df.columns.tolist()
columns = columns[0:1] + columns[-3:] + columns[1:]
del columns[-3:]
df = df[columns]
TempImportanceScore = df['Importance Score']
df = df.drop(columns=['Importance Score'])
df.insert(loc=0, column='Importance Score', value=TempImportanceScore)

#Reorder columns and export final, merged dataframe as CSV

df = df.reset_index()
df = df.set_index('Importance Score').sort_index(axis = 0, ascending = True)
df.to_csv(path)



In [16]:
# Create a new dataframe with only important songs (songs in this dataframe that has not broken the USTop20 Chart yet)
# Important Songs are songs that have broken into at least one city's Top20 Charts but has not broken into the US Top20 Chart
# delete the column 'Rank in USTop20 Chart

filename = 'MostImportantSongsinUS.csv'
path = pathtoDFolder + filename

important_df = df[(df['Number of Cities where Song Broke Top Chart']>0) & (df['Broke US Top Chart'] == False)]
important_df = important_df.reset_index()
important_df = (important_df
                    .set_index(['Importance Score'])
                    .sort_index(axis = 0, ascending = True))
important_df = important_df.drop(columns = ['Rank in Top US Chart'])
important_df.head()

#Export to CSV 
important_df.to_csv(path)

In [17]:
df = important_df.reset_index()
df = df.set_index(['Song Name', 'Artist']).head()

#Set index to Song name and artist, keep only columns of cities, and extract top 5 Most Important Songs 
df_new = df.loc[:, 'Aberdeen-SD':'Yuma']
df_new

,,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,Anchorage,Asheville,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
Don't Be Cruel,Elvis Presley,NaN,37.0,16.0,NaN,28.0,37.0,NaN,17.0,25.0,2.0,...,NaN,NaN,18.0,NaN,NaN,NaN,39.0,NaN,NaN,NaN
THATS WHAT I WANT,Lil Nas X,NaN,NaN,NaN,19.0,13.0,NaN,NaN,37.0,26.0,18.0,...,NaN,32.0,NaN,NaN,27.0,NaN,34.0,10.0,19.0,NaN
Pussy Power,Gunna & Drake,NaN,NaN,23.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,...,38.0,NaN,25.0,NaN,19.0,NaN,NaN,NaN,NaN,16.0
When I R.I.P.,Labrinth,40.0,39.0,13.0,NaN,9.0,NaN,31.0,NaN,16.0,16.0,...,NaN,24.0,34.0,32.0,NaN,NaN,33.0,NaN,NaN,31.0
Beautiful Lies,Yung Bleu & Kehlani,NaN,NaN,NaN,12.0,NaN,31.0,36.0,NaN,NaN,NaN,...,39.0,NaN,38.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN


In [18]:
# Create a dictionary out of the dataframe to extract all cities that each song is in its Top20 Chart
# Rotate columns and axis to make the songs/artists keys in the dictionary 
df_new = df_new.T

# Total dataframe in dictionary form 
# keys = song/artist tuple 
# Value = dictionary where key = column name and value = element from dataframe
dict = df_new.to_dict()
dict

{("Don't Be Cruel", 'Elvis Presley'): {'Aberdeen-SD': nan,
  'Aberdeen-WA': 37.0,
  'Abilene': 16.0,
  'Akron': nan,
  'Albany': 28.0,
  'Albuquerque': 37.0,
  'Allentown': nan,
  'Anaheim': 17.0,
  'Anchorage': 25.0,
  'Asheville': 2.0,
  'Atlanta': nan,
  'Aurora': nan,
  'Austin': 4.0,
  'Bakersfield': nan,
  'Baltimore': 39.0,
  'Barnstable': 14.0,
  'Baton-Rouge': 21.0,
  'Bellingham': 2.0,
  'Belvidere': nan,
  'Bend': 15.0,
  'Billings': nan,
  'Birmingham': nan,
  'Bismarck': 40.0,
  'Boise': 27.0,
  'Boston': 16.0,
  'Brentwood-NY': 18.0,
  'Bridgeport': 12.0,
  'Brockton': 8.0,
  'Brooklyn': nan,
  'Brownsville': nan,
  'Buffalo': nan,
  'Cape-Coral': nan,
  'Casper': nan,
  'Cedar-Rapids': nan,
  'Centreville': nan,
  'Charleston': 9.0,
  'Charlotte': nan,
  'Cheyenne': nan,
  'Chicago': 18.0,
  'Chico': 2.0,
  'Cincinnati': nan,
  'Clarksville-TN': nan,
  'Cleveland': nan,
  'Colorado-Springs': 15.0,
  'Columbia': nan,
  'Columbus': 23.0,
  'Concord-CA': 24.0,
  'Coram-NY':

In [19]:
#Populate a new dictionary where its keys = songs/artist and values = all cities that the song is in its top20 chart
SongCitiesDict = {}
ColumnName = 'List of Cities where Song Broke Top Chart'
# Extract song/artist name 
for Song in dict:
    CityList = []
    CityDict = dict.get(Song)
    for City in CityDict:
        if pd.isna(CityDict.get(City)) == False:
            CityList.append(City)
    SongCitiesDict[Song] = CityList
    
    #Turn CityList into a string to insert into the original df dataframe 
    CityList = ', '.join([str(city) for city in CityList])
    df.loc[Song, ColumnName] = CityList
df
    

,,Importance Score,Number of Cities where Song Broke Top Chart,Broke US Top Chart,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,...,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma,List of Cities where Song Broke Top Chart
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
Don't Be Cruel,Elvis Presley,128.79,88.0,False,NaN,37.0,16.0,NaN,28.0,37.0,NaN,...,NaN,18.0,NaN,NaN,NaN,39.0,NaN,NaN,NaN,"Aberdeen-WA, Abilene, Albany, Albuquerque, Ana..."
THATS WHAT I WANT,Lil Nas X,136.10,82.0,False,NaN,NaN,NaN,19.0,13.0,NaN,NaN,...,32.0,NaN,NaN,27.0,NaN,34.0,10.0,19.0,NaN,"Akron, Albany, Anaheim, Anchorage, Asheville, ..."
Pussy Power,Gunna & Drake,141.76,76.0,False,NaN,NaN,23.0,NaN,17.0,NaN,NaN,...,NaN,25.0,NaN,19.0,NaN,NaN,NaN,NaN,16.0,"Abilene, Albany, Atlanta, Aurora, Austin, Balt..."
When I R.I.P.,Labrinth,142.36,75.0,False,40.0,39.0,13.0,NaN,9.0,NaN,31.0,...,24.0,34.0,32.0,NaN,NaN,33.0,NaN,NaN,31.0,"Aberdeen-SD, Aberdeen-WA, Abilene, Albany, All..."
Beautiful Lies,Yung Bleu & Kehlani,145.54,72.0,False,NaN,NaN,NaN,12.0,NaN,31.0,36.0,...,NaN,38.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN,"Akron, Albuquerque, Allentown, Atlanta, Bakers..."


In [20]:
#Reorder Columns
columns = df.columns.tolist()
columns = columns[0:2]+ columns[-1:] + columns[2:]
del columns[-1:]
df = df[columns]
df

,,Importance Score,Number of Cities where Song Broke Top Chart,List of Cities where Song Broke Top Chart,Broke US Top Chart,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma
Song Name,Artist,,,,,,,,,,,,,,,,,,,,,
Don't Be Cruel,Elvis Presley,128.79,88.0,"Aberdeen-WA, Abilene, Albany, Albuquerque, Ana...",False,NaN,37.0,16.0,NaN,28.0,37.0,...,NaN,NaN,18.0,NaN,NaN,NaN,39.0,NaN,NaN,NaN
THATS WHAT I WANT,Lil Nas X,136.10,82.0,"Akron, Albany, Anaheim, Anchorage, Asheville, ...",False,NaN,NaN,NaN,19.0,13.0,NaN,...,NaN,32.0,NaN,NaN,27.0,NaN,34.0,10.0,19.0,NaN
Pussy Power,Gunna & Drake,141.76,76.0,"Abilene, Albany, Atlanta, Aurora, Austin, Balt...",False,NaN,NaN,23.0,NaN,17.0,NaN,...,38.0,NaN,25.0,NaN,19.0,NaN,NaN,NaN,NaN,16.0
When I R.I.P.,Labrinth,142.36,75.0,"Aberdeen-SD, Aberdeen-WA, Abilene, Albany, All...",False,40.0,39.0,13.0,NaN,9.0,NaN,...,NaN,24.0,34.0,32.0,NaN,NaN,33.0,NaN,NaN,31.0
Beautiful Lies,Yung Bleu & Kehlani,145.54,72.0,"Akron, Albuquerque, Allentown, Atlanta, Bakers...",False,NaN,NaN,NaN,12.0,NaN,31.0,...,39.0,NaN,38.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN


In [21]:
# Export to CSV
path = '/Users/zacharywong/github/zacharywong2023/AtlanticRecords/Top5ImportantSongs.csv'
df = df.reset_index()
df.to_csv(path, index = False)

In [22]:
df = df.fillna('')

spreadsheet_id = '1WYvfPFW6n2hOCZ-2_pTT0hjJOj5vGHbwjLnjzDqXhSM'
spreadsheet_name = 'Top 5 Important Songs of the Day'

sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.get_worksheet(0)
worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1WYvfPFW6n2hOCZ-2_pTT0hjJOj5vGHbwjLnjzDqXhSM',
 'updatedRange': "'Top 5 Important Songs of the Day'!A1:GZ6",
 'updatedRows': 6,
 'updatedColumns': 208,
 'updatedCells': 1248}

In [23]:
PauseEmailCellLocation = 'B12'
EmailPause = readinValue(spreadsheet_id, PauseEmailCellLocation)
#print(EmailPause)

if (EmailPause == 'N'):
    subject = 'Top 5 Most Important Songs of the Day'
    text = "Hi Jake and the team - attached is today's CSV attachment with the 5 most important songs of the day you should look out for."
    sender_email = "zacharywongdatascience"
    receiver_email = 'zachary.j.wong.23@dartmouth.edu'
    password = ''
    pathtoPassword = '/Users/zacharywong/Documents/ApplicationPassword-Secret/ApplicationPassword.txt'
    with open (pathtoPassword, 'r') as file:
        password = file.read()
    filename = 'Top5ImportantSongs-' + str(date.today()) +'.csv'

    filepath = '/Users/zacharywong/github/zacharywong2023/AtlanticRecords/Top5ImportantSongs.csv'

    msg = MIMEMultipart()
    msg ["From"] = sender_email
    msg ["To"] = receiver_email
    msg ["Subject"] = subject
    msg.attach(MIMEText(text, "plain"))

    with open (filepath, 'rb') as file:
        msg.attach(MIMEApplication(file.read(), Name=filename))

    msg['Content Disposition'] = "attachment; filename=Top5ImportantSongs"
    content = msg.as_string()

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, content)
